In [1]:
import sys
import os
import cutlet

from jreadability import compute_readability


# 获取当前 Notebook 所在目录的上层目录
upper_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
# 将上层目录添加到 sys.path
sys.path.append(upper_dir)

# 导入上层目录中的 Python 文件（假设文件名为 module.py）
import notebook_common


import jaconv

from importlib import reload

reload(notebook_common)
from notebook_common import (
    # ------------------function------------------
    # read_excel,
    to_csv_sig,
    rd_csv_sig,
    # ------------------2400 word------------------
    # r_word_2400_xlsx,
    # w_word_2400_csv,
    # w_word_2400_with_guide_csv,
    w_word_2400_without_guide_csv,
    # ------------------书面语相关------------------
    # 解析书面语正则
    # pattern_table_written,
    # pattern_td_written,
    # r_written_freq_nlt_with_enter_txt,
    # w_written_freq_nlt_with_enter_csv,
    w_written_freq_nlt_sumed_csv,
    # ------------------口语相关------------------
    # 解析口头语正则
    # pattern_table_spoken,
    # pattern_td_spoken,
    # r_spoken_freq_nlb_with_enter_txt,
    # w_spoken_freq_nlb_with_enter_csv,
    # w_spoken_freq_nlb_with_enter_filtered_csv,
    # w_spoken_freq_nlb_with_enter_filtered_hrt_csv,
    w_spoken_freq_nlb_sumed_csv,
    # ------------------合并书面语和口语词频------------------
    w_merged_freq_sumed_csv,
    w_raw_merged_freq_sumed_csv,
    w_word_raw_merged_freq_sumed_csv,
    # ------------------公共python库------------------
    pd,
    np,
    p,
    l,
)

p("Import common module")

Import common module


In [33]:
# 原始基础词频表
df_merged_raw_word_agg = pd.read_csv(
    w_word_raw_merged_freq_sumed_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    # keep_default_na=False,
    # index_col=["headword"]
)
df_merged_raw_word_agg.rename(columns={'headword':'spell'},inplace=True)
df_merged_raw_word_agg.head()

,word_id,spell,frequency,spoken_freq,written_freq,fused_freq,pron_count,spell_type
0,100000,の,1,57221425.0,686025.0,43219589.0,1,0
1,100001,に,2,35286930.0,0.0,23536382.0,1,0
2,100002,を,3,33707106.0,0.0,22482640.0,1,0
3,100003,は,4,31279365.0,0.0,20863336.0,1,0
4,100004,が,5,30705135.0,0.0,20480325.0,1,0


In [34]:
# 读取base_meaning 词性词义表  3800条
df_base_meaning = rd_csv_sig("D:/Dropbox/06.wanjuan/99.tmp/normal_merged_result.csv")
df_base_meaning["meaning_id"] = df_base_meaning["meaning_id"].astype(str)
df_base_meaning["final_str"] = (
    df_base_meaning["meaning_id"]
    + "|"
    + df_base_meaning["spell"]
    + "|"
    + df_base_meaning["pron"]
    + "|"
    + df_base_meaning["pos"]
    + "|"
    + df_base_meaning["meaning"]
)
df_base_meaning=df_base_meaning[['final_str']]
df_base_meaning

,final_str
0,1|ああ|ああ|感叹词|感叹/表示惊讶或感叹
1,2|愛|あい|名词|爱/深厚的感情
2,3|愛|あい|名词|慈爱/关怀
3,4|合図|あいず|名词|信号/示意
4,5|合図|あいず|名词|暗号/约定信号
...,...
3853,3854|指|ゆび|名词|指/指向
3854,3855|礼|れい|名词|礼节/礼貌
3855,3856|礼|れい|名词|谢礼/酬谢
3856,3857|別れる|わかれる|动2|分开/分离


In [35]:
# 关联精简词义
df_simple_meaning = pd.read_csv(
    "D:/Dropbox/06.wanjuan/99.tmp/simple_meaning.csv",
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
)

df_simple_meaning = df_simple_meaning[["spell", "simple_meaning"]]
df_merged_raw_word_agg["simple_meaning"] = df_merged_raw_word_agg["spell"].map(
    df_simple_meaning.set_index("spell")["simple_meaning"]
)


# 先将原始10万词条和高考2400词条作outer join
# df_outer_joiner = pd.merge(
#     df_merged_raw_word_agg, df_simple_meaning, on="spell", how="outer", indicator="check"
# )


# 设置word_type 1为高中2400词汇， 0为默认类型
# df_outer_joiner["word_type"] = 0
# df_outer_joiner.loc[
#     df_outer_joiner["check"].isin(["both", "right_only"]), "word_type"
# ] = 1

# # 1：已有词性词义例句数据 0：默认
# df_outer_joiner["has_data"] = 0
# df_outer_joiner.loc[df_outer_joiner["check"].isin(["both"]), "has_data"] = 1

# null_word_id_indices = df_outer_joiner["word_id"].isnull()
# num_null_word_ids = null_word_id_indices.sum()

# if num_null_word_ids > 0:
#     start_id = 210000
#     new_ids = np.arange(start_id, start_id + num_null_word_ids)
#     df_outer_joiner.loc[null_word_id_indices, "word_id"] = new_ids


# 读取10万精简词义
df_100000_simple_meaning = pd.read_csv(
    "D:/Dropbox/06.wanjuan/99.tmp/100000-simple-meaning.csv",
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    # index_col=["spell"],
)


df_merged_raw_word_agg["simlpe_meaning_more"] = df_merged_raw_word_agg["spell"].map(
    df_100000_simple_meaning.set_index("spell")["meaning"]
)


df_merged_raw_word_agg["meaning_origin"] = np.where(
    df_merged_raw_word_agg["simple_meaning"].isna(), "dp", "grok"
)
df_merged_raw_word_agg["simple_meaning"] = df_merged_raw_word_agg[
    "simple_meaning"
].fillna(df_merged_raw_word_agg["simlpe_meaning_more"])


df_merged_raw_word_agg.sort_values(["frequency"], inplace=True)
del df_merged_raw_word_agg["simlpe_meaning_more"]

df_merged_raw_word_agg.rename(columns={"fused_freq": "mix_freq"}, inplace=True)
katsu = cutlet.Cutlet()
df_merged_raw_word_agg["roman"] = df_merged_raw_word_agg["spell"].apply(katsu.romaji)

# 增加字段，判断拼写属于7个类型中的哪一个
# 1. 汉字：只包含汉字，例如「日本語」。
# 2. 汉字和平假名：包含汉字和平假名，例如「漢字ひらがな」。
# 3. 汉字和片假名：包含汉字和片假名，例如「漢字カタカナ」。
# 4. 汉字、平假名和片假名：包含汉字、平假名和片假名，例如「漢字ひらカタ」。
# 5. 纯平假名：只包含平假名，例如「ひらがな」。
# 6. 纯片假名：只包含片假名，例如「カタカナ」。
# 7. 平假名和片假名混合：包含平假名和片假名，例如「ひらカタ」。

kanji_range = r"[\u4E00-\u9FCF]"
hiragana_range = r"[\u3040-\u309F]"
katakana_range = r"[\u30A0-\u30FF]"

# 检测字符类型
# has_kanji = bool(re.search(kanji_range, text))
# has_hiragana = bool(re.search(hiragana_range, text))
# has_katakana = bool(re.search(katakana_range, text))

df_merged_raw_word_agg["has_kanji"] = df_merged_raw_word_agg["spell"].str.contains(
    kanji_range
)
df_merged_raw_word_agg["hiragana_range"] = df_merged_raw_word_agg["spell"].str.contains(
    hiragana_range
)
df_merged_raw_word_agg["katakana_range"] = df_merged_raw_word_agg["spell"].str.contains(
    katakana_range
)

df_merged_raw_word_agg["code"] = (
    (df_merged_raw_word_agg["has_kanji"].astype(int) * 4)
    | (df_merged_raw_word_agg["hiragana_range"].astype(int) * 2)
    | (df_merged_raw_word_agg["katakana_range"].astype(int))
)


value_map = {
    4: 1,  # 1. 汉字：只包含汉字，例如「日本語」。(100)
    6: 2,  # 2. 汉字和平假名：包含汉字和平假名，例如「漢字ひらがな」。(110)
    5: 3,  # 3. 汉字和片假名：包含汉字和片假名，例如「漢字カタカナ」。(101)
    7: 4,  # 4. 汉字、平假名和片假名：包含汉字、平假名和片假名，例如「漢字ひらカタ」。(111)
    2: 5,  # 5. 纯平假名：只包含平假名，例如「ひらがな」。(010)
    1: 6,  # 6. 纯片假名：只包含片假名，例如「カタカナ」。(001)
    3: 7,  # 7. 平假名和片假名混合：包含平假名和片假名，例如「ひらカタ」。(011)
    0: 8,  # 8. 其它 (000)
}

df_merged_raw_word_agg["kanji_type"] = df_merged_raw_word_agg["code"].map(value_map)


to_csv_sig(
    df_merged_raw_word_agg, "D:/Dropbox/06.wanjuan/99.tmp/df_base_freq_new22_to_del.csv"
)

l("ok")
# to_csv_sig(df_base_freq,'D:/Dropbox/06.wanjuan/99.tmp/base_freq_with_simple_meaning.csv')
# df_base_freq

----------------------------------------ok----------------------------------------


In [201]:
# 将38000条例句关联到发音,词性,词义
df_exp_38000 = rd_csv_sig("D:/Dropbox/06.wanjuan/99.tmp/df_exp_38000.csv")
df_exp_38000

,meaning_id,exp_text,exp_ch,jlpt_level
0,1,<b>ああ</b>、なんて美しい朝日でしょう。,啊，多么美丽的朝阳啊。,N5
1,1,<b>ああ</b>、びっくりしました。,啊，吓了我一跳。,N5
2,1,<b>ああ</b>、やっと終わりました。,啊，终于结束了。,N4
3,1,<b>ああ</b>、こんなに寒いなんて。,啊，没想到这么冷。,N4
4,1,<b>ああ</b>、忘れてしまいました。,啊，我忘记了。,N3
...,...,...,...,...
38532,3858,突然<b>別れさせられた</b>のでショックです。,突然被迫分手，我很震惊。,N3
38533,3858,<b>別れている</b>間も連絡を取ります。,即使分手期间也会保持联系。,N2
38534,3858,<b>別れざるを得ない</b>理由がありました。,有不得不分手的理由。,N2
38535,3858,心から愛した人と<b>別れる</b>のは悲しいです。,和真心爱的人分手很悲伤。,N1


In [202]:
# 过滤meaning_id长度大于4的非法行
df_exp_38000 = df_exp_38000[
    (df_exp_38000["meaning_id"].str.len() < 5)
    & (df_exp_38000["jlpt_level"].str.len() < 5)
]


In [203]:
# 关联词义表
df_pos_meaning_3800 = rd_csv_sig("D:/Dropbox/06.wanjuan/99.tmp/normal_merged_result.csv")
df_pos_meaning_3800["meaning_id"] = df_pos_meaning_3800["meaning_id"].astype(str)
df_exp = pd.merge(
    df_exp_38000,
    df_pos_meaning_3800[
        ["meaning_id", "word_id", "pron_id", "spell", "pron", "pos", "meaning"]
    ],
    on="meaning_id",
    how="left",
    indicator="check",
)
start_index = 100000
df_exp.index = range(start_index, start_index + len(df_exp))
df_exp.index.name = "exp_id"
df_exp.reset_index(inplace=True)

order = [
    "exp_id",
    "meaning_id",
    "word_id",
    "pron_id",
    "spell",
    "pron",
    "pos",
    "meaning",
    "exp_text",
    "exp_ch",
    "jlpt_level",
]
df_exp = df_exp[order]

to_csv_sig(df_exp, "D:/Dropbox/06.wanjuan/99.tmp/df_normal_exp_38000.csv")
df_exp.head()

,exp_id,meaning_id,word_id,pron_id,spell,pron,pos,meaning,exp_text,exp_ch,jlpt_level
0,100000,1,103383.0,10338310.0,ああ,ああ,感叹词,感叹/表示惊讶或感叹,<b>ああ</b>、なんて美しい朝日でしょう。,啊，多么美丽的朝阳啊。,N5
1,100001,1,103383.0,10338310.0,ああ,ああ,感叹词,感叹/表示惊讶或感叹,<b>ああ</b>、びっくりしました。,啊，吓了我一跳。,N5
2,100002,1,103383.0,10338310.0,ああ,ああ,感叹词,感叹/表示惊讶或感叹,<b>ああ</b>、やっと終わりました。,啊，终于结束了。,N4
3,100003,1,103383.0,10338310.0,ああ,ああ,感叹词,感叹/表示惊讶或感叹,<b>ああ</b>、こんなに寒いなんて。,啊，没想到这么冷。,N4
4,100004,1,103383.0,10338310.0,ああ,ああ,感叹词,感叹/表示惊讶或感叹,<b>ああ</b>、忘れてしまいました。,啊，我忘记了。,N3


In [204]:
df_exp["exp_text"] = df_exp["exp_text"].str.strip()
df_exp["exp_text"] = df_exp["exp_text"].str.replace(r"^-", "", regex=True).str.strip()
df_exp["exp_ch"] = df_exp["exp_ch"].str.strip()
df_exp["exp_ch"] = df_exp["exp_ch"].str.replace(r"^-", "", regex=True).str.strip()


df_exp["exp_text_no_flags"] = df_exp["exp_text"].str.replace(r"<[^>]+>", "", regex=True)
# df_exp['readability_point']=df_exp["exp_text_no_flags"].apply(compute_readability)

to_csv_sig(df_exp, "D:/Dropbox/06.wanjuan/99.tmp/df_normal_exp_38000.csv")

df_exp.head()

,exp_id,meaning_id,word_id,pron_id,spell,pron,pos,meaning,exp_text,exp_ch,jlpt_level,exp_text_no_flags,readability_point
0,100000,1,103383.0,10338310.0,ああ,ああ,感叹词,感叹/表示惊讶或感叹,<b>ああ</b>、なんて美しい朝日でしょう。,啊，多么美丽的朝阳啊。,N5,ああ、なんて美しい朝日でしょう。,7.703429
1,100001,1,103383.0,10338310.0,ああ,ああ,感叹词,感叹/表示惊讶或感叹,<b>ああ</b>、びっくりしました。,啊，吓了我一跳。,N5,ああ、びっくりしました。,8.332000
2,100002,1,103383.0,10338310.0,ああ,ああ,感叹词,感叹/表示惊讶或感叹,<b>ああ</b>、やっと終わりました。,啊，终于结束了。,N4,ああ、やっと終わりました。,8.332000
3,100003,1,103383.0,10338310.0,ああ,ああ,感叹词,感叹/表示惊讶或感叹,<b>ああ</b>、こんなに寒いなんて。,啊，没想到这么冷。,N4,ああ、こんなに寒いなんて。,7.703429
4,100004,1,103383.0,10338310.0,ああ,ああ,感叹词,感叹/表示惊讶或感叹,<b>ああ</b>、忘れてしまいました。,啊，我忘记了。,N3,ああ、忘れてしまいました。,7.576000


In [205]:
df_exp["readability_point"] = round(df_exp["readability_point"], 2)

to_csv_sig(df_exp, "D:/Dropbox/06.wanjuan/99.tmp/df_normal_exp_38000_2.csv")
l('ok')


----------------------------------------ok----------------------------------------


In [206]:
df_normal_exp_38000_tmp = rd_csv_sig("D:/Dropbox/06.wanjuan/99.tmp/df_normal_exp_38000.csv")
df_normal_exp_38000_tmp["readability_point"] = round(
    df_normal_exp_38000_tmp["readability_point"], 3
)
to_csv_sig(df_normal_exp_38000_tmp, "D:/Dropbox/06.wanjuan/99.tmp/df_normal_exp_38000_3.csv")